In [1]:
#pip install ipynb --upgrade

In [2]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

os.chdir("..")
import Models
os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras import Input, Model

from tcn import TCN, tcn_full_summary

In [3]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [4]:
settings = load_yaml('mock_settings.yaml')

In [5]:
dataframe_btcusd = pd.read_csv('../btcusd.csv')

In [6]:
dataframe_btcusd.set_index('Date', inplace=True)
dataframe_btcusd.head()

,Label,Open,High,Low,Volume,stocK,stocD,stocSD,Momentum,ROC,...,RSI,OBV,MA,BIAS6,PSY12,ASY5,ASY4,ASY3,ASY2,ASY1
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-03,4100.11,4508.50,4714.76,4298.33,691.216198,72.056081,79.434454,87.853957,422.79,10.346549,...,56.229448,1360.246980,4619.232,-0.022727,58.3,-0.344754,-0.254078,-1.558371,-3.488469,0.822610
2017-09-04,4366.47,4505.00,4527.49,3972.51,1394.644614,45.485613,62.399271,76.691393,84.11,2.094373,...,40.473369,-34.397634,4528.226,-0.095440,58.3,-2.104849,-3.545762,-5.494957,-4.342662,-9.507934
2017-09-05,4619.77,4106.97,4484.99,3603.00,1228.938157,57.137832,58.226509,66.686744,326.47,8.080941,...,50.250949,1194.540523,4456.542,-0.029946,66.7,-1.577788,-2.547690,-0.797071,-1.606912,6.294110
2017-09-06,4691.61,4366.49,4662.87,4335.26,807.363726,76.094717,59.572721,60.066167,505.76,12.293602,...,57.414561,2001.904249,4413.514,0.030337,66.7,-0.910352,0.811947,0.808393,5.966556,5.639002
2017-09-07,4282.80,4619.77,4788.59,4438.19,500.429975,81.471198,71.567916,63.122382,375.60,8.702482,...,59.208599,2502.334224,4457.408,0.051963,66.7,0.958175,0.992067,4.492067,3.591045,1.543089


In [7]:
dataframe = pd.read_csv('../nvda.csv')

In [8]:
dataframe.set_index('Date', inplace=True)
dataframe.head()

,Label,Open,High,Low,Volume,stocK,stocD,stocSD,Momentum,ROC,...,RSI,OBV,MA,BIAS6,PSY12,ASY5,ASY4,ASY3,ASY2,ASY1
Date,,,,,,,,,,,,,,,,,,,,,
1999-02-17,1.682292,1.708333,1.729167,1.625000,1693200.0,53.246749,66.460777,58.270866,-0.010417,-0.624998,...,51.775678,-6704400.0,1.661458,0.010058,58.3,1.569432,2.218203e+00,0.210305,-2.454482,-5.505978
1999-02-18,1.739583,1.708333,1.729167,1.635417,1767600.0,59.740246,63.203456,64.743252,0.020833,1.253914,...,53.690848,-4936800.0,1.694792,0.010428,66.7,2.086581,5.477515e-01,-1.116291,-1.972943,1.560092
1999-02-19,1.750000,1.666667,1.770833,1.645833,1884000.0,74.025981,62.337659,64.000631,0.156250,9.868421,...,57.673414,-3052800.0,1.713542,0.021928,66.7,1.107976,-2.220446e-16,-0.199005,2.454482,3.348872
1999-02-22,1.833333,1.770833,1.791667,1.656250,5131200.0,76.623374,70.129867,65.223661,0.135417,8.387094,...,58.374347,2078400.0,1.715625,0.017668,66.7,0.119403,-2.220446e-16,1.835326,1.972943,0.597014
1999-02-23,1.979167,1.791667,1.869792,1.687500,3452400.0,91.139258,80.596205,71.021243,0.343750,23.076922,...,63.571668,5530800.0,1.732292,0.056528,66.7,0.930401,2.539495e+00,2.865963,2.624509,4.652004


In [16]:
dataframe.columns

Index(['Label', 'Open', 'High', 'Low', 'Volume', 'stocK', 'stocD', 'stocSD',
       'Momentum', 'ROC', 'LWR', 'AOosci', 'Disp5', 'Disp10', 'OSCP', 'CCI',
       'RSI', 'OBV', 'MA', 'BIAS6', 'PSY12', 'ASY5', 'ASY4', 'ASY3', 'ASY2',
       'ASY1'],
      dtype='object')

In [9]:
data_all_linreg = features.split(dataframe, 'linreg', settings['ensemble']['models'][0]['linreg'])
data_all = features.split(dataframe, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [10]:
#  WORKING
linreg = Models.train_model(data_all_linreg, 'linreg', settings['ensemble']['models'][0])

In [11]:
linreg

{'model': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 'predictions': array([0.54675293, 0.56044388, 0.54888916, ..., 0.62610626, 0.52565765,
        0.55870438])}

In [12]:
#  WORKING
lstm = Models.train_model(data_all, 'lstm', settings['ensemble']['models'][1]['lstm'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 3265 samples, validate on 1088 samples
Epoch 1/7
3265/3265 [==============================] - 20s 6ms/sample - loss: 0.0063 - val_loss: 0.0018
Epoch 2/7
3265/3265 [==============================] - 19s 6ms/sample - loss: 0.0026 - val_loss: 0.0015
Epoch 3/7
3265/3265 [==============================] - 17s 5ms/sample - loss: 0.0022 - val_loss: 6.0438e-04
Epoch 4/7
3265/3265 [==============================] - 18s 5ms/sample - loss: 0.0021 - val_loss: 0.0026
Epoch 5/7
3265/3265 [==============================] - 17s 5ms/sample - loss: 0.0021 - val_loss: 0.0012
Epoch 6/7
3265/3265 [==============================] - 16s 5ms/sample - loss: 0.0020 - val_loss: 6.3842e-04
Epoch 7/7
3265/3265 [==============================] - 16s 5ms/sample - loss: 0.0019 - val_loss: 9.1632e-04


In [13]:
lstm

{'model': <tensorflow.python.keras.engine.sequential.Sequential at 0x7ff556300550>,
 'predictions': array([[0.5158551 ],
        [0.52409273],
        [0.52524155],
        ...,
        [0.55848426],
        [0.5566244 ],
        [0.5460373 ]], dtype=float32)}

In [14]:
#  WORKING
tcn = Models.train_model(data_all, 'tcn', settings['ensemble']['models'][3]['tcn'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25, 1)]           0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 25, 64), (None, 2 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 25, 64), (None, 2 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 25, 64), (None, 2 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 25, 64), (None, 2 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 25, 64), (None, 2 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 25, 64), (None, 2 16512 

In [15]:
tcn

{'model': <tensorflow.python.keras.engine.training.Model at 0x7ff5561d5250>,
 'predictions': array([[0.448194  ],
        [0.52676594],
        [0.50804764],
        ...,
        [2.746056  ],
        [4.2162666 ],
        [2.876979  ]], dtype=float32)}